# BOM_wet_dry_times_loop_sites

This notebook uses gridded daily BoM data to determine wet and dry seasons based off the available range of gridded BoM data (1987 - 2013). 

This notebok combines all of the processing steps into a single cell so that the whole process can be looped for each of the case study sites. To see what each step does in more detail, refer to "BOM_wet_dry_times", which includes plotting and print-to-screen outputs.

** Code dependencies **
- csv file containing the bounding boxes for the case study site/s

** Accompanying code **
- BOM_wet_dry_times.ipynb - steps through inputs and outputs for this code. See this accompanying notebook for explanation of the functionality of this code.


Created by Claire Krause. November 2016. Datacube version 1.1.13. Python v3.

In [1]:
# Import the librarys we need for our analysis
%matplotlib inline
import datacube
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import scipy.stats
import pandas
import csv
import logging
logging.getLogger('datacube.storage.storage').setLevel(logging.ERROR)

dc = datacube.Datacube(app='dc-BOMrainfall')
dc

Datacube<index=Index<db=PostgresDb<engine=Engine(postgresql://cek156@130.56.244.227:6432/datacube)>>>

## Import the list of case study sites and bounding boxes.¶

In [2]:
# Set up the case study bounding box (to make the file smaller and avoid memory errors)
# Read in a csv file with all case study bounding boxes
names = pandas.read_csv('./case_study_sites.csv', delimiter = ',')
print(names)
x = len(names.index)
iterable = list(range(0,x))

         Name  minlat  maxlat  minlon  maxlon
0  Blackwood2  -34.10  -32.70  116.40  118.40
1     Garford  -29.90  -29.23  133.15  134.70
2    Mandora1  -21.10  -18.87  120.90  127.00
3   Mandora1a  -20.03  -18.88  120.90  123.30
4   Murchison  -27.95  -25.49  114.75  119.30
5         Ord  -15.93  -15.05  128.30  129.29
6      TiTree  -23.04  -21.67  133.18  134.24
7    Daintree  -16.39  -16.09  145.21  145.42
8       Laura  -15.85  -14.43  143.42  144.89
9  Blackwood1  -34.40  -33.58  115.00  116.40


## Now loop through all the sites in our file to find the 5th and 95th percentile dry and wet years using BOM gridded daily rainfall

To see each of these steps in more detail, see "BOM_wet_dry_times"

In [3]:
# Set up your loop so that it analyses each of the sites in the file.

for num in iterable:
    Studysite = names.ix[num]
    print('Working on ' + Studysite.Name)

    # Now set up the query with the chosen study site
    query = {'lat': (names.maxlat[num], names.minlat[num]), 
             'lon': (names.minlon[num], names.maxlon[num]), 
             'time': ('1987-01-01','2014-01-01') }

    # Grab bom_rainfall_grids from the datacube
    Studysite_rain = dc.load(product = 'bom_rainfall_grids', **query)

    # Resample to 3 monthly data, starting with December
    monthly_avg = Studysite_rain.resample('QS-DEC', dim = 'time', how = 'mean')

    # Create an average across our whole bounding box - i.e. over lat and lon
    monthly_avg_all = monthly_avg.mean(dim = ('latitude', 'longitude'))
    monthly_avg_all

    # Find the 95th percentile
    threshold95 = monthly_avg_all.reduce(np.percentile, dim = 'time', q = 95)

    # Find the 5th percentile
    threshold5 = monthly_avg_all.reduce(np.percentile, dim = 'time', q = 5)

    # Note that the resampling we did means that each season is labelled according to its first day 
    # e.g. 1990-12-01 is Summer starting in Dec 1990.
    wet_seasons = monthly_avg_all.where(monthly_avg_all.rainfall>=threshold95).dropna(dim = 'time')
    dry_seasons = monthly_avg_all.where(monthly_avg_all.rainfall<=threshold5).dropna(dim = 'time')

    # Change the format so that it's YYYY-MM-DD
    dry = np.array(dry_seasons.time, dtype = 'datetime64[D]')
    wet = np.array(wet_seasons.time, dtype = 'datetime64[D]')

    # Now add these into a new csv file for later
    site_name = names.Name[num]
    site = [site_name,dry,wet]

    # If this is the first site, make a new file, otherwise, append the existing file
    if num == 0:
        with open('/g/data/p25/cek156/BOM_site_rainfall.csv','w') as csvFile:
            writer = csv.writer(csvFile)
            header = ['name','dry','wet']
            writer.writerow(header)
            writer.writerow(site)
    else:
        with open('/g/data/p25/cek156/BOM_site_rainfall.csv','a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(site)

Working on Blackwood2
Working on Garford
Working on Mandora1
Working on Mandora1a
Working on Murchison
Working on Ord
Working on TiTree
Working on Daintree
Working on Laura
Working on Blackwood1
